In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.widgets import Slider
import os
import pandas as pd
from time import time

In [2]:
def apply_hu_threshold_for_pe_slice(npy_slice, lower_bound, upper_bound):
    mask = (npy_slice < lower_bound) | (npy_slice > upper_bound)
    npy_slice[mask] = 0
    return npy_slice

In [3]:
def show_slice_window(slice, level, window):
    max_val = level + window / 2
    min_val = level - window / 2
    return slice.clip(min_val, max_val)

In [4]:
npy_files = [f for f in os.listdir('npy') if f.endswith('.npy')]
npy_array = np.load('npy/' + npy_files[0])

In [5]:
csv_dir = 'csv/'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
print(csv_files)

['Labels.csv', 'Demographics.csv']


In [6]:
data_dict = {}
plt.ion()

for npy_file in npy_files:
    idx = int(npy_file.split('.')[0])
    print(idx)
    patient_data = {}
    for csv_file in csv_files:
        csv_data = pd.read_csv(os.path.join(csv_dir, csv_file))
        matching_data = csv_data[csv_data['idx'] == idx]
        if "pe_type" in matching_data:
            if matching_data["pe_type"].isna().any():
                matching_data["pe_type"].fillna("negative", inplace=True)
        
        # Consolidate race columns
        race_columns = ['Asian', 'Black', 'Native American', 'Other', 'Pacific Islander', 'Unknown_race', 'White']
        race_value = next((race for race in race_columns if race in matching_data.columns and matching_data.iloc[0][race] == 1), None)
        if race_value:
            matching_data['Race'] = race_value
            for race in race_columns:
                if race in matching_data.columns:
                    matching_data.drop(race, axis=1, inplace=True)

        # Consolidate smoker columns
        if 'SMOKER_Y' in matching_data.columns and matching_data.iloc[0]['SMOKER_Y'] == 1:
            matching_data['Smoker'] = 'Yes'
        elif 'SMOKER_N' in matching_data.columns and matching_data.iloc[0]['SMOKER_N'] == 1:
            matching_data['Smoker'] = 'No'
        if 'SMOKER_Y' in matching_data.columns:
            matching_data.drop('SMOKER_Y', axis=1, inplace=True)
        if 'SMOKER_N' in matching_data.columns:
            matching_data.drop('SMOKER_N', axis=1, inplace=True)
            
        # Consolidate gender columns:
        if 'Male' in matching_data.columns and matching_data.iloc[0]['Male'] == 1:
            matching_data['Gender'] = 'Male'
        elif 'Female' in matching_data.columns and matching_data.iloc[0]['Female'] == 1:
            matching_data['Gender'] = 'Female'
        if 'Male' in matching_data.columns:
            matching_data.drop('Male', axis=1, inplace=True)
        if 'Female' in matching_data.columns:
            matching_data.drop('Female', axis=1, inplace=True)
        
        if not matching_data.empty:
            patient_data.update(matching_data.to_dict('records')[0])
            
    if patient_data:
        data_dict[idx] = patient_data


0
2000


/var/folders/s6/8j8nyd6530n0yb8h5skx35s40000gn/T/ipykernel_89347/2953663828.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_data["pe_type"].fillna("negative", inplace=True)
/var/folders/s6/8j8nyd6530n0yb8h5skx35s40000gn/T/ipykernel_89347/2953663828.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_data['Race'] = race_value
/var/folders/s6/8j8nyd6530n0yb8h5skx35s40000gn/T/ipykernel_89347/2953663828.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [7]:
for npy_file in npy_files:
    idx = int(npy_file.split('.')[0])
    print(npy_file)
    npy_array = np.load('npy/' + npy_file)
    initial_slice = npy_array.shape[0] // 2
    initial_lower_bound = -1000
    initial_upper_bound = 200
    initial_window = 2000
    initial_level = 100

    fig, ax = plt.subplots()
    fig.suptitle(npy_file)
    im = plt.imshow(apply_hu_threshold_for_pe_slice(npy_array[initial_slice].copy(), initial_lower_bound, initial_upper_bound), cmap='gray')
    plt.subplots_adjust(bottom=0.45, right=0.7)

    ax_slider_slice = plt.axes([0.2, 0.1, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider_slice = Slider(ax_slider_slice, 'Slice', 0, npy_array.shape[0] - 1, valinit=initial_slice, valstep=1)

    ax_slider_lower = plt.axes([0.2, 0.2, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider_lower = Slider(ax_slider_lower, 'Lower HU', -2000, 0, valinit=initial_lower_bound)

    ax_slider_upper = plt.axes([0.2, 0.15, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider_upper = Slider(ax_slider_upper, 'Upper HU', 0, 2000, valinit=initial_upper_bound)

    ax_slider_window = plt.axes([0.2, 0.3, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider_window = Slider(ax_slider_window, 'Window', 1, 2000, valinit=initial_window)

    ax_slider_level = plt.axes([0.2, 0.35, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider_level = Slider(ax_slider_level, 'Level', -1000, 1000, valinit=initial_level)

    last_time = time()

    text_box = plt.axes([0.7, 0.4, 0.18, 0.55], frameon=True)
    text_box.axis('off')

    def display_patient_data(idx):
        data = data_dict.get(idx, {})
        info_str = '\n'.join([f"{key}: {val}" for key, val in data.items()])
        text_box.clear()
        text_box.axis('off')
        text_box.text(0, 1, info_str, va='top')
        plt.draw()

    def update(val):
        global last_time
        current_time = time()
        if current_time - last_time < 0.1:
            return
        last_time = current_time

        slice_index = int(slider_slice.val)
        lower_bound = int(slider_lower.val)
        upper_bound = int(slider_upper.val)
        window = int(slider_window.val)
        level = int(slider_level.val)
        filtered_slice = apply_hu_threshold_for_pe_slice(npy_array[slice_index].copy(), lower_bound, upper_bound)
        windowed_slice = show_slice_window(filtered_slice, level, window)
        
        im.set_data(windowed_slice)
        fig.canvas.draw_idle()
        
    display_patient_data(idx)
    slider_slice.on_changed(update)
    slider_lower.on_changed(update)
    slider_upper.on_changed(update)
    slider_window.on_changed(update)
    slider_level.on_changed(update)

    plt.show(block=False)
    plt.show()


0.npy
2000.npy


: 